In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env

In [2]:
url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)
df.head()

,entry
0,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
1,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
2,97.105.19.58 - - [16/Apr/2019:19:34:44 +0000] ...
3,97.105.19.58 - - [16/Apr/2019:19:34:46 +0000] ...
4,97.105.19.58 - - [16/Apr/2019:19:34:48 +0000] ...


In [3]:
df['entry'][0].split()

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [3]:
parts = df['entry'][0].split()
output = {}

output['ip'] = parts[0]
output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
pd.Series(output)

ip                   97.105.19.58
timestamp    16/Apr/2019 19:34:42
dtype: object

In [4]:
# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)

In [5]:
df = df.entry.apply(parse_log_entry)
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [6]:
df['size_mb'] = df['size'] / 1024 / 1024
df.timestamp = pd.to_datetime(df.timestamp)
df = df.set_index('timestamp')
df.head()

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb
timestamp,,,,,,,,
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396
2019-04-16 19:34:44,97.105.19.58,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472
2019-04-16 19:34:46,97.105.19.58,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0,0.486377
2019-04-16 19:34:48,97.105.19.58,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0,0.488246


### In this exercises, I will explore request_path and user_agent

#### * request_path

In [7]:
df.request_path.value_counts()

/api/v1/sales?page=2                 709
/api/v1/items                        464
/api/v1/items?page=2                 291
/api/v1/items?page=3                 219
/api/v1/stores                       162
                                    ... 
/api/v1/items?page=0                   1
/api/v1/stores?page=999                1
/api/v1/stores?page=2                  1
/api/v1/stores?page=666                1
/api/v1/items/api/v1/items?page=3      1
Name: request_path, Length: 218, dtype: int64

In [8]:
path_df = pd.DataFrame(df.request_path.value_counts(dropna=False)).reset_index().\
                rename(columns={'index': 'path', 'request_path': 'count'})
path_df.head()

,path,count
0,/api/v1/sales?page=2,709
1,/api/v1/items,464
2,/api/v1/items?page=2,291
3,/api/v1/items?page=3,219
4,/api/v1/stores,162


In [75]:
path_df2 = pd.DataFrame((df.request_path.value_counts(dropna=False))/df.request_path.count()).reset_index().\
                rename(columns={'index': 'path', 'request_path': 'proba'})
path_df2.head()

,path,proba
0,/api/v1/sales?page=2,0.050737
1,/api/v1/items,0.033205
2,/api/v1/items?page=2,0.020824
3,/api/v1/items?page=3,0.015672
4,/api/v1/stores,0.011593


In [76]:
path_df = path_df.merge(path_df2, on='path')
path_df.head()

,path,count,proba
0,/api/v1/sales?page=2,709,0.050737
1,/api/v1/items,464,0.033205
2,/api/v1/items?page=2,291,0.020824
3,/api/v1/items?page=3,219,0.015672
4,/api/v1/stores,162,0.011593


In [46]:
path_df

,path,count,proba
0,/api/v1/sales?page=2,709,0.050737
1,/api/v1/items,464,0.033205
2,/api/v1/items?page=2,291,0.020824
3,/api/v1/items?page=3,219,0.015672
4,/api/v1/stores,162,0.011593
...,...,...,...
213,/api/v1/items?page=0,1,0.000072
214,/api/v1/stores?page=999,1,0.000072
215,/api/v1/stores?page=2,1,0.000072
216,/api/v1/stores?page=666,1,0.000072


In [ ]:
# prob(A|B) = prob(A & B)/prob(B)
# A = path, B = ip

In [9]:
path_pab = df.groupby(['ip','request_path']).size()/len(df)
path_pab

ip              request_path           
173.173.113.51  /                          0.000072
                /api/v1/items              0.004580
                /api/v1/items?page=2       0.004437
                /api/v1/items?page=3       0.004437
                /api/v1/sales              0.000572
                                             ...   
97.105.19.58    /api/v1/stores?page=666    0.000072
                /api/v1/stores?page=999    0.000072
                /api/v1items               0.000072
                /documentation             0.006655
                /favicon.ico               0.001002
Length: 816, dtype: float64

In [34]:
ip_pb = df.groupby('ip').size()/len(df)
ip_pb

ip
173.173.113.51    0.075784
24.26.242.9       0.001503
3.88.129.158      0.000072
3.92.201.136      0.000072
34.207.64.242     0.000072
34.229.70.250     0.000072
35.174.209.2      0.000072
35.175.171.137    0.000143
45.23.250.16      0.000072
52.87.230.102     0.000143
52.90.165.200     0.000072
52.91.30.150      0.000072
54.145.52.184     0.000072
54.172.14.223     0.000072
68.201.219.223    0.001503
70.121.214.34     0.000143
72.181.105.81     0.017604
72.181.113.170    0.043867
97.105.19.58      0.858595
dtype: float64

In [35]:
path_pab/ip_pb

ip              request_path           
173.173.113.51  /                          0.000944
                /api/v1/items              0.060434
                /api/v1/items?page=2       0.058546
                /api/v1/items?page=3       0.058546
                /api/v1/sales              0.007554
                                             ...   
97.105.19.58    /api/v1/stores?page=666    0.000083
                /api/v1/stores?page=999    0.000083
                /api/v1items               0.000083
                /documentation             0.007751
                /favicon.ico               0.001167
Length: 816, dtype: float64

In [36]:
path_given_ip = (df.groupby('ip').request_path.value_counts(normalize=True)
                 .rename('proba_path_given_ip').reset_index())
path_given_ip.head(20)

,ip,request_path,proba_path_given_ip
0,173.173.113.51,/api/v1/items,0.060434
1,173.173.113.51,/api/v1/items?page=2,0.058546
2,173.173.113.51,/api/v1/items?page=3,0.058546
3,173.173.113.51,/api/v1/stores,0.050992
4,173.173.113.51,/api/v1/sales,0.007554
5,173.173.113.51,/api/v1/sales?page=10,0.004721
6,173.173.113.51,/api/v1/sales?page=11,0.004721
7,173.173.113.51,/api/v1/sales?page=12,0.004721
8,173.173.113.51,/api/v1/sales?page=13,0.004721
9,173.173.113.51,/api/v1/sales?page=14,0.004721


In [13]:
path_given_ip[path_given_ip.proba_path_given_ip > 0.1]

,ip,request_path,proba_path_given_ip
189,24.26.242.9,/api/v1/items?page=1,0.333333
190,24.26.242.9,/api/v1/sales?page=1,0.333333
191,24.26.242.9,/api/v1/stores?page=1,0.333333
192,3.88.129.158,/api/v1/items,1.000000
193,3.92.201.136,/,1.000000
194,34.207.64.242,/favicon.ico,1.000000
195,34.229.70.250,/favicon.ico,1.000000
196,35.174.209.2,/favicon.ico,1.000000
197,35.175.171.137,/,0.500000
198,35.175.171.137,/api/v1/items,0.500000


In [14]:
df = df.reset_index().merge(path_given_ip, on=['ip', 'request_path'], how='left').fillna(value=0).set_index('timestamp')

In [15]:
df.head()

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_path_given_ip
timestamp,,,,,,,,,
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753,0.004417
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396,0.028505
2019-04-16 19:34:44,97.105.19.58,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472,0.004417
2019-04-16 19:34:46,97.105.19.58,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0,0.486377,0.004417
2019-04-16 19:34:48,97.105.19.58,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0,0.488246,0.004417


In [20]:
df[df.proba_path_given_ip == 1]

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_path_given_ip
timestamp,,,,,,,,,
2019-04-16 19:39:14,3.92.201.136,GET,/,HTTP/1.1,200,42,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.000040,1.0
2019-04-16 19:39:15,52.90.165.200,GET,/favicon.ico,HTTP/1.1,200,162,Slackbot 1.0 (+https://api.slack.com/robots),0.000154,1.0
2019-04-16 19:40:32,3.88.129.158,GET,/api/v1/items,HTTP/1.1,200,3561,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.003396,1.0
2019-04-16 19:40:33,34.207.64.242,GET,/favicon.ico,HTTP/1.1,200,162,Slackbot 1.0 (+https://api.slack.com/robots),0.000154,1.0
2019-04-16 19:40:36,54.172.14.223,GET,/api/v1/,HTTP/1.1,200,162,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.000154,1.0
2019-04-16 19:40:37,34.229.70.250,GET,/favicon.ico,HTTP/1.1,200,162,Slackbot 1.0 (+https://api.slack.com/robots),0.000154,1.0
2019-04-16 21:22:22,54.145.52.184,GET,/favicon.ico,HTTP/1.1,200,162,Slackbot 1.0 (+https://api.slack.com/robots),0.000154,1.0
2019-04-16 21:38:57,35.174.209.2,GET,/favicon.ico,HTTP/1.1,200,162,Slackbot 1.0 (+https://api.slack.com/robots),0.000154,1.0
2019-04-17 02:14:27,52.91.30.150,GET,/,HTTP/1.1,200,42,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.000040,1.0


In [25]:
df[df.request_path == '/favicon.ico']

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_path_given_ip
timestamp,,,,,,,,,
2019-04-16 19:38:16,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.001167
2019-04-16 19:39:15,52.90.165.200,GET,/favicon.ico,HTTP/1.1,200,162,Slackbot 1.0 (+https://api.slack.com/robots),0.000154,1.000000
2019-04-16 19:40:33,34.207.64.242,GET,/favicon.ico,HTTP/1.1,200,162,Slackbot 1.0 (+https://api.slack.com/robots),0.000154,1.000000
2019-04-16 19:40:37,34.229.70.250,GET,/favicon.ico,HTTP/1.1,200,162,Slackbot 1.0 (+https://api.slack.com/robots),0.000154,1.000000
2019-04-16 19:44:04,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.001167
2019-04-16 19:44:16,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.001167
2019-04-16 19:44:24,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.001167
2019-04-16 19:44:28,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.001167
2019-04-16 19:44:38,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.001167


#### * user_agent

In [30]:
agent_df = pd.DataFrame(df.user_agent.value_counts(dropna=False)).reset_index().\
                rename(columns={'index': 'agent', 'user_agent': 'count'})
agent_df.head()

,agent,count
0,python-requests/2.21.0,12001
1,python-requests/2.20.1,1911
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,34
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,8
4,Slackbot-LinkExpanding 1.0 (+https://api.slack...,7


In [31]:
agent_df2 = pd.DataFrame((df.user_agent.value_counts(dropna=False))/df.user_agent.count()).reset_index().\
                rename(columns={'index': 'agent', 'user_agent': 'proba'})
agent_df2.head()

,agent,proba
0,python-requests/2.21.0,0.858809
1,python-requests/2.20.1,0.136754
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.002433
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,0.000572
4,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.000501


In [32]:
agent_df = agent_df.merge(agent_df2, on='agent')
agent_df.head()

,agent,count,proba
0,python-requests/2.21.0,12001,0.858809
1,python-requests/2.20.1,1911,0.136754
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,34,0.002433
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,8,0.000572
4,Slackbot-LinkExpanding 1.0 (+https://api.slack...,7,0.000501


In [ ]:
# prob(A|B) = prob(A & B)/prob(B)
# A = agent, B = ip

In [33]:
pab_agent = df.groupby(['ip', 'user_agent']).size()/len(df)
pab_agent

ip              user_agent                                                                                                               
173.173.113.51  python-requests/2.21.0                                                                                                       0.075784
24.26.242.9     python-requests/2.21.0                                                                                                       0.001503
3.88.129.158    Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)                                                                   0.000072
3.92.201.136    Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)                                                                   0.000072
34.207.64.242   Slackbot 1.0 (+https://api.slack.com/robots)                                                                                 0.000072
34.229.70.250   Slackbot 1.0 (+https://api.slack.com/robots)                                                    

In [37]:
pab_agent/ip_pb

ip              user_agent                                                                                                               
173.173.113.51  python-requests/2.21.0                                                                                                       1.000000
24.26.242.9     python-requests/2.21.0                                                                                                       1.000000
3.88.129.158    Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)                                                                   1.000000
3.92.201.136    Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)                                                                   1.000000
34.207.64.242   Slackbot 1.0 (+https://api.slack.com/robots)                                                                                 1.000000
34.229.70.250   Slackbot 1.0 (+https://api.slack.com/robots)                                                    

In [38]:
agent_given_ip = (df.groupby('ip').user_agent.value_counts(normalize=True)
                 .rename('proba_agent_given_ip').reset_index())
agent_given_ip.head(20)

,ip,user_agent,proba_agent_given_ip
0,173.173.113.51,python-requests/2.21.0,1.000000
1,24.26.242.9,python-requests/2.21.0,1.000000
2,3.88.129.158,Slackbot-LinkExpanding 1.0 (+https://api.slack...,1.000000
3,3.92.201.136,Slackbot-LinkExpanding 1.0 (+https://api.slack...,1.000000
4,34.207.64.242,Slackbot 1.0 (+https://api.slack.com/robots),1.000000
5,34.229.70.250,Slackbot 1.0 (+https://api.slack.com/robots),1.000000
6,35.174.209.2,Slackbot 1.0 (+https://api.slack.com/robots),1.000000
7,35.175.171.137,Slackbot-LinkExpanding 1.0 (+https://api.slack...,1.000000
8,45.23.250.16,python-requests/2.21.0,1.000000
9,52.87.230.102,Slackbot 1.0 (+https://api.slack.com/robots),0.500000


In [39]:
df = df.reset_index().merge(agent_given_ip, on=['ip', 'user_agent'], how='left').fillna(value=0).set_index('timestamp')
df.head()

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_path_given_ip,proba_agent_given_ip
timestamp,,,,,,,,,,
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753,0.004417,0.838306
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396,0.028505,0.838306
2019-04-16 19:34:44,97.105.19.58,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472,0.004417,0.838306
2019-04-16 19:34:46,97.105.19.58,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0,0.486377,0.004417,0.838306
2019-04-16 19:34:48,97.105.19.58,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0,0.488246,0.004417,0.838306


In [42]:
df[df.proba_agent_given_ip < 0.05]

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_path_given_ip,proba_agent_given_ip
timestamp,,,,,,,,,,
2019-04-16 19:38:16,97.105.19.58,GET,/api/V1/HiZach!,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.000083,0.002167
2019-04-16 19:38:16,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.001167,0.002167
2019-04-16 19:44:04,97.105.19.58,GET,/api/v1/stores?page=0,HTTP/1.1,200,1328,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.001266,0.000167,0.002167
2019-04-16 19:44:04,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.001167,0.002167
2019-04-16 19:44:16,97.105.19.58,GET,/api/v1/stores?page=1,HTTP/1.1,200,1328,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.001266,0.004001,0.002167
2019-04-16 19:44:16,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.001167,0.002167
2019-04-16 19:44:24,97.105.19.58,GET,/api/v1/stores?page=1,HTTP/1.1,200,1328,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.001266,0.004001,0.002167
2019-04-16 19:44:24,97.105.19.58,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.000154,0.001167,0.002167
2019-04-16 19:44:28,97.105.19.58,GET,/api/v1/stores?page=2,HTTP/1.1,200,1328,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.001266,0.000083,0.002167
